# <font color='red' style='font-size: 30px;'>Dataset Netflix e IMDb</font>
<hr style='border: 2px solid black;'>


## Importando as principais Bibliotecas

<hr style='border: red;'>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline
import plotly.graph_objects as go

import warnings

warnings.filterwarnings('ignore') # ou warnings.filterwarnings(action='once')

## <font color='red' style='font-size: 30px;'>Importando as bases de dados</font>

### Base de dados do Netflix

In [ ]:

df = pd.read_csv("../input/netflix-shows/netflix_titles.csv")
df.columns

In [ ]:
df.info()

In [ ]:
df.isna().sum()

### Precisamos fazer algumas alterações no Dataframe
<ul color='red' style='font-size: 18px; line-height: 2; text-align: justify;'>

- Excluir a coluna 'Cast' e 'Description'
- Criar a coluna 'Year_added' apartir da coluna 'Date_added'
- Criar uma nova coluna "Season_count" no dataset para fixar a duração das series de TV,atráves de uma função Lambda
- Cortar da coluna "duration" os objetos que apresentam duração em temporadas



In [ ]:
df.drop(['cast','description', 'director'], axis=1, inplace=True)
df

In [ ]:
df["date_added"] = pd.to_datetime(df['date_added'])
df['year_added'] = df['date_added'].dt.year

In [ ]:
df['season_count'] = df.apply(lambda x : x['duration'].split(" ")[0] if "Season" in x['duration'] else "", axis = 1)
df['duration_movie'] = df.apply(lambda x : x['duration'].split(" ")[0] if "Season" not in x['duration'] else "", axis = 1)
df.head()

In [ ]:
df.info()

### Extraindo o primeiro gênero listado e criando a coluna 'principal_genero'

In [ ]:
df['principal_genero'] = df['listed_in'].apply(lambda x: x.split(", ")[0])
df['principal_genero'].head()

In [ ]:
df.loc[20, "listed_in"]

In [ ]:
df.loc[20, "principal_genero"]

In [ ]:
df.isna().sum()

### <font color='black' style='font-size: 30px;'>Alterando as classificações etárias para melhor entendimento</font>

In [ ]:
ratings_ages = {
    'TV-PG': 'Older Kids',
    'TV-MA': 'Adults',
    'TV-Y7-FV': 'Older Kids',
    'TV-Y7': 'Older Kids',
    'TV-14': 'Teens',
    'R': 'Adults',
    'TV-Y': 'Kids',
    'NR': 'Adults',
    'PG-13': 'Teens',
    'TV-G': 'Kids',
    'PG': 'Older Kids',
    'G': 'Kids',
    'UR': 'Adults',
    'NC-17': 'Adults'
}

In [ ]:
df['target_ages'] = df['rating'].replace(ratings_ages)
df['target_ages'].unique()

In [ ]:
df.columns

## Transformando dados categóricos e númericos

In [ ]:
df['type'] = pd.Categorical(df['type'])

df['target_ages'] = pd.Categorical(df['target_ages'], categories=['Kids', 'Older Kids', 'Teens', 'Adults'])

df['duration_movie'] = pd.to_numeric(df['duration_movie']) 

df['year_added'] = pd.to_numeric(df['year_added'])

df.info()

## Limpando dados faltantes

In [ ]:
df.isna().sum()

### Usando o método mode() e utilizando a moda para preencher os valores ausentes

In [ ]:
df['country'] = df['country'].fillna(df['country'].mode()[0])
df['target_ages'] = df['target_ages'].fillna(df['target_ages'].mode()[0])

In [ ]:
df['year_added'] = df['year_added'].fillna(df['year_added'].mode()[0])

In [ ]:
df.isna().sum()

In [ ]:
df.drop(['date_added','rating', 'duration', 'listed_in',], axis=1, inplace=True)


In [ ]:
df.describe()

In [ ]:
df.isna().sum()

### Criando a coluna do 'principal_country' que acomodará o principal país produtor

In [ ]:
df['principal_country'] = df['country'].apply(lambda x: x.split(",")[0])
df['principal_country'].head()

## Separando o DataFrame de Movies e Tv Show's

In [ ]:
filmes_df=df[df['type']=='Movie']
tv_show_df=df[df['type']=='TV Show']


In [ ]:
filme = len(filmes_df)
tv_show = len(tv_show_df)

## Começando a Análise Exploratória dos Dados

### Como se divide o dataset entre Movies e Tv shows?

In [ ]:
df_types_ = df['type'].value_counts()
df_types_= pd.DataFrame(df_types_)
df_types_ = df_types_.reset_index()
df_types_

In [ ]:
df_types_ = df_types_.rename({'index': 'type',
                          'type': 'quantidade'}, axis = 1)

df_types_

In [ ]:
df['type'].value_counts(normalize=True)

In [ ]:
fig = px.bar(df_types_, x="type", y="quantidade", color='type', title="Quantidade por tipo")
fig.show()

### Observações:
<ul color='red' style='font-size: 18px; line-height: 2; text-align: justify;'>

- Os filmes representam 69% do Dataset da Netflix

### Como se comporta a produção de Movies e Tv Shows?


- released_year_df será  todas as produções lançadas a partir de 2008 agrupadas por ano de lançamento e tipo.
- added_year_df será  todas as produções adicionadas ao catálogo da Netdlix a partir de 2008 agrupadas por ano de lançamento e tipo.

In [ ]:
released_year_df = df.loc[df['release_year'] > 2007].groupby(['release_year', 'type']).agg({'show_id': 'count'}).reset_index()
released_year_df

In [ ]:
added_year_df = df.loc[df['year_added'] > 2007].groupby(['year_added', 'type']).agg({'show_id': 'count'}).reset_index()
added_year_df

## Linha temporal de lançamento e adição de filmes a partir de 2008

In [ ]:

fig = go.Figure()

fig.add_trace(go.Scatter( 
    x=released_year_df.loc[released_year_df['type'] == 'Movie']['release_year'], 
    y=released_year_df.loc[released_year_df['type'] == 'Movie']['show_id'],
    mode='lines+markers',
    name='Movie: Released Year',
    marker_color='green',
))

fig.add_trace(go.Scatter( 
    x=released_year_df.loc[released_year_df['type'] == 'TV Show']['release_year'], 
    y=released_year_df.loc[released_year_df['type'] == 'TV Show']['show_id'],
    mode='lines+markers',
    name='TV Show: Released Year',
    marker_color='Darkgreen',
))

fig.add_trace(go.Scatter( 
    x=added_year_df.loc[added_year_df['type'] == 'TV Show']['year_added'], 
    y=added_year_df.loc[added_year_df['type'] == 'TV Show']['show_id'],
    mode='lines+markers',
    name='TV Show: Added Year',
    marker_color='blue',
))

fig.add_trace(go.Scatter( 
    x=added_year_df.loc[added_year_df['type'] == 'Movie']['year_added'], 
    y=added_year_df.loc[added_year_df['type'] == 'Movie']['show_id'],
    mode='lines+markers',
    name='Movie: Added Year',
    marker_color='orange',
))


### Os principais países produtores de conteúdo


- Os EUA sozinhos representam 43% do Dataset
- Para melhor analizarmos iremos dividir o dataset com e sem os EUA

In [ ]:
df['principal_country'].value_counts(normalize=True)

In [ ]:
main_pais_eua = df['principal_country'].value_counts().head(11).reset_index()

In [ ]:
fig = px.bar(main_pais_eua, x="index", y="principal_country", 
             color ='principal_country',color_continuous_scale= 'deep', 
             title="10 maiores de produtores de conteúdo da Netflix")
fig.show()

In [ ]:
main_pais = df['principal_country'].value_counts().head(11).reset_index()[1:]

In [ ]:
fig = px.bar(main_pais, x="index", y="principal_country", 
             color ='principal_country',color_continuous_scale= 'deep', 
             title="10 maiores de produtores de conteúdo da Netflix sem os EUA")
fig.show()

### Como se divide o Dataset entre as Classificações etárias? No geral e entre 2012 e 2020?

- 'teste_df' representa o dataset 'df' agrupado pela classificação etária e pelo tipo
- 'teste_antes' representa o dataset 'df' agrupado pela classificação etária e pelo tipo adicionados em 2012
- 'teste_depois' representa o dataset 'df' agrupado pela classificação etária e pelo tipo adicionados em 2020

In [ ]:
teste_df = df.groupby(['target_ages','type']).agg({'show_id': 'count'}).reset_index()

In [ ]:
teste_antes = df.loc[df['year_added'] == 2013].groupby(['target_ages', 'type']).agg({'show_id': 'count'}).reset_index()
teste_depois = df.loc[df['year_added'] == 2020].groupby(['target_ages', 'type']).agg({'show_id': 'count'}).reset_index()


In [ ]:
teste_df

In [ ]:
g = sns.catplot(
    data= teste_df, kind="bar",
    x="target_ages", y="show_id", hue="type",
    ci="sd", palette="Reds", alpha=.6, height=6
)

g.despine(left=True)
g.set_axis_labels("target ages", "Número de produções")
g.legend.set_title("Legenda:")


In [ ]:
g = sns.catplot(
    data= teste_antes, kind="bar",
    x="target_ages", y="show_id", hue="type",
    ci="sd", palette="Reds", alpha=.6, height=6
)

g.despine(left=True)
g.set_axis_labels("target ages 2013", "Número de produções 2013")
g.legend.set_title("Legenda:")

In [ ]:
g = sns.catplot(
    data= teste_depois, kind="bar",
    x="target_ages", y="show_id", hue="type",
    ci="sd", palette="Reds", alpha=.6, height=6
)

g.despine(left=True)
g.set_axis_labels("target ages 2020", "Número de produções em 2020")
g.legend.set_title("Legenda:")

### Observações

- Os primeiros TV shows adicionados ao catálogo eram das categorias teens e Adults
- É próximo a proporção de TV shows e Movies no cenário de 2020 em comparaçã ao quadro geral
- Nos dois cenários é bastante próxima a produção de Movies e TV shows da categoria Kids

## Principais gêneros listados no Dataset

In [ ]:
df.columns

In [ ]:
df['principal_genero'].value_counts(normalize=True).head(10)

In [ ]:
main_genero = pd.DataFrame(df['principal_genero'].value_counts().head(10).reset_index())
main_genero


In [ ]:
fig = px.bar(main_genero, x="index", y="principal_genero", 
             color ='principal_genero',color_continuous_scale= 'Reds', 
             title="10 principais gêneros listados no Dataset")
fig.show()

### Observações

- Drama e Comédia são os gêneros mais famosos no dataset
- O gênero de comédia também pode ser considerado no gênero 'Stand-up Comedy'

## Entre Movie e Tv Show's:

In [ ]:
filmes_df.head()


In [ ]:
tv_show_df.head()

In [ ]:
filme_pais = filmes_df['principal_country'].value_counts().head(11).reset_index()
filme_pais

In [ ]:
fig = px.bar(filme_pais, x="index", y="principal_country", 
             color ='principal_country',color_continuous_scale= 'deep', 
             title="10 maiores de produtores de filme da Netflix")
fig.show()

### Tirando os EUA

In [ ]:
fig = px.bar(filme_pais[2:], x="index", y="principal_country", 
             color ='principal_country',color_continuous_scale= 'deep', 
             title="10 maiores de produtores de filme da Netflix")
fig.show()

## Como se distribuiu no tempo as classificações etárias?

- 'target_year_df' representa o dataset agrupado pelas produções adicionadas em 2012 e pelas suas classificações etárias
- 'target_year_release' representa o dataset agrupado pelo ano de lançamento e pela classificação etárias das produções
- Foi produzido dois gráficos para se avaliar o comportamento das produções de acordo com sua categória etária ao longo do tempo

In [ ]:
target_year_df = df.loc[df['year_added'] > 2013].groupby(['year_added', 'target_ages']).agg({'show_id': 'count'}).reset_index()
target_year_df

In [ ]:

fig = go.Figure()

fig.add_trace(go.Scatter( 
    x=target_year_df.loc[target_year_df['target_ages'] == 'Kids']['year_added'], 
    y=target_year_df.loc[target_year_df['target_ages'] == 'Kids']['show_id'],
    mode='lines+markers',
    name='Kids: Added Year',
    marker_color='green',
))

fig.add_trace(go.Scatter( 
    x=target_year_df.loc[target_year_df['target_ages'] == 'Older Kids']['year_added'], 
    y=target_year_df.loc[target_year_df['target_ages'] == 'Older Kids']['show_id'],
    mode='lines+markers',
    name=' Older Kids: Added Year',
    marker_color='Darkgreen',
))

fig.add_trace(go.Scatter( 
    x=target_year_df.loc[target_year_df['target_ages'] == 'Teens']['year_added'], 
    y=target_year_df.loc[target_year_df['target_ages'] == 'Teens']['show_id'],
    mode='lines+markers',
    name='Teens: Added Year',
    marker_color='blue',
))

fig.add_trace(go.Scatter( 
    x=target_year_df.loc[target_year_df['target_ages'] == 'Adults']['year_added'], 
    y=target_year_df.loc[target_year_df['target_ages'] == 'Adults']['show_id'],
    mode='lines+markers',
    name='Adults: Added Year',
    marker_color='orange',
))


In [ ]:
target_year_release = df.groupby(['release_year', 'target_ages']).agg({'show_id': 'count'}).reset_index()
target_year_release

In [ ]:

fig = go.Figure()

fig.add_trace(go.Scatter( 
    x=target_year_release.loc[target_year_release['target_ages'] == 'Kids']['release_year'], 
    y=target_year_release.loc[target_year_release['target_ages'] == 'Kids']['show_id'],
    mode='lines+markers',
    name='Kids: Added Year',
    marker_color='green',
    
    
    ))

fig.add_trace(go.Scatter( 
    x=target_year_release.loc[target_year_release['target_ages'] == 'Older Kids']['release_year'], 
    y=target_year_release.loc[target_year_release['target_ages'] == 'Older Kids']['show_id'],
    mode='lines+markers',
    name=' Older Kids: Ano de Lançamento',
    marker_color='Darkgreen',
))

fig.add_trace(go.Scatter( 
    x=target_year_release.loc[target_year_release['target_ages'] == 'Teens']['release_year'], 
    y=target_year_release.loc[target_year_release['target_ages'] == 'Teens']['show_id'],
    mode='lines+markers',
    name='Teens: Ano de Lançamento',
    marker_color='blue',
))

fig.add_trace(go.Scatter( 
    x=target_year_release.loc[target_year_release['target_ages'] == 'Adults']['release_year'], 
    y=target_year_release.loc[target_year_release['target_ages'] == 'Adults']['show_id'],
    mode='lines+markers',
    name='Adults: Ano de Lançamento',
    marker_color='orange',
))


fig.show()

In [ ]:
filmes_df

In [ ]:
filmes_df.columns



### Crescimento ou decrescimento marginal da produção de Movies e TV show?

In [ ]:
filmes_df_ = filmes_df.loc[filmes_df['release_year']> 2007].groupby('year_added').agg({'show_id' : 'count'})

In [ ]:
filmes_df_.diff().plot()

In [ ]:
filmes_df_1 = filmes_df.loc[filmes_df['release_year']> 2008].groupby('release_year').agg({'show_id' : 'count'})

In [ ]:
filmes_df_1.diff().plot()

In [ ]:
tv_show_df

tv_show_df_ = tv_show_df.loc[tv_show_df['release_year']> 2007].groupby('year_added').agg({'show_id' : 'count'})

In [ ]:
tv_show_df_

In [ ]:
tv_show_df_.diff().plot()

## Diferença entre ano de lançamento e ano de adição no catálogo do Netflix

In [ ]:
df_teste_1 = df[['title','release_year','year_added']]

In [ ]:
df_diff = df_teste_1.set_index('title').diff(axis=1)

In [ ]:
df_diff

In [ ]:
df_diff

In [ ]:
df_treino = df[['title','release_year','year_added']]

##  Dos filmes lançados a partir de 2008 qual demorou mais anos para ser adicionado no catálogo da Netflix?

In [ ]:
target_df = df.loc[df['release_year'] == 2008]

In [ ]:
target_df = target_df[['title','release_year','year_added']]

In [ ]:
target_df_diff = target_df.set_index('title').diff(axis=1)

In [ ]:
target_df_diff.max()

### Os 10 filmes que mais demoraram para ser adicionados no catálogo da Netflix

In [ ]:
target_df_diff['year_added'].sort_values(ascending=False).head(10)

In [ ]:
df.set_index('title').loc['Jaane Tu... Ya Jaane Na']

# <font color='red' style='font-size: 30px;'>Juntando os datasets do Netflix e do IMDb</font>
<hr style='border: 2px solid black;'>


## Anexando o Dataset do IMDB

- Utilizando apenas as colunas : "weighted_average_vote","total_votes" do arquivo "ratings.csv"

- Utilizando apenas as colunas : 'title','year','genre' do arquivo "movies.csv"


In [ ]:
imdb_rating= pd.read_csv("../input/imdb-extensive-dataset/IMDb ratings.csv",usecols=["weighted_average_vote","total_votes"])
imdb_rating['total_votes'] 


In [ ]:
imdb_titles=pd.read_csv("../input/imdb-extensive-dataset/IMDb movies.csv",usecols=['title','year','genre'])
imdb_titles.columns


### Criando o DataFrame ratings_imdb

In [ ]:
ratings_imdb=pd.DataFrame({'Title':imdb_titles['title'],
                     'Release Year':imdb_titles['year'],
                     'Rating': imdb_rating['weighted_average_vote'],
                     'Total_votes': imdb_rating['total_votes'],
                     'Genre': imdb_titles['genre']})

In [ ]:
ratings_imdb.sort_values('Total_votes',ascending=False).reset_index()

In [ ]:
len(ratings_imdb)

In [ ]:
len(df)

### Juntando com o método 'merge' os dois dataframes

In [ ]:
joint_data=ratings_imdb.merge(df,left_on="Title",right_on='title',how='inner')
len(joint_data)

In [ ]:
joint_data.isna().sum()

In [ ]:
joint_data.set_index('Title').sort_values('Rating', ascending=False)

### Separando os Dataframes de Filmes e TV Shows

In [ ]:
filmes_joint=joint_data[joint_data['type']=='Movie']
tv_show_joint=joint_data[joint_data['type']=='TV Show']


In [ ]:
len(tv_show_joint)

In [ ]:
len(filmes_joint)

In [ ]:
tv_show_joint.set_index('Title').sort_values('Rating', ascending=False).head(10)

In [ ]:
filmes_joint.columns

### Excluindo colunas que não serão utilizadas

In [ ]:
filmes_joint.drop(['title','country','Release Year', 'Genre', 'type',], axis=1, inplace=True)

In [ ]:
filmes_joint.drop([ 'season_count'], axis=1, inplace=True)

In [ ]:
filmes_joint.set_index('Title').sort_values('Rating', ascending=False)

In [ ]:
filmes_joint.describe().round(2).T

### Atribuindo classes ao número de votantes 

- No dataset há filmes avaliados por mais de 1 milhão de usuários;
- Por outro há filmes avaliados por apenas 100 usuários;
- Seria justo avaliar as notas de produções com diferenças tão discrepantes de avaliações?

In [ ]:
valor_minimo = filmes_joint['Total_votes'].min()
valor_maximo = filmes_joint['Total_votes'].max()


In [ ]:
classes = [valor_minimo, 50000, 500000, 1000000, valor_maximo]

In [ ]:
rotulos = ['votacao_D', 'votacao_C', 'votacao_B', 'votacao_A']

In [ ]:
filmes_joint['votacao_label'] = pd.cut(x=filmes_joint['Total_votes'], bins=classes, labels=rotulos, include_lowest=True)

In [ ]:
filmes_joint.set_index('Title').sort_values('Rating', ascending=False)

In [ ]:
filmes_joint.groupby('votacao_label').describe().round(2)

### Tv show

In [ ]:

valor_minimo = tv_show_joint['Total_votes'].min()
valor_maximo = tv_show_joint['Total_votes'].max()

In [ ]:
valor_minimo

In [ ]:
valor_maximo

In [ ]:
tv_show_joint.describe().round(2).T

In [ ]:
classes_tv = [valor_minimo, 50000, 300000,valor_maximo]

In [ ]:
rotulos_tv = ['votacao_C', 'votacao_B', 'votacao_A']

In [ ]:
tv_show_joint['votacao_label'] = pd.cut(x=tv_show_joint['Total_votes'], bins=classes_tv, labels=rotulos_tv, include_lowest=True)

In [ ]:
tv_show_joint.set_index('Title').sort_values('Rating', ascending=False)

## Análise Exploratória dos dados

In [ ]:
filmes_joint.set_index('Title').sort_values('Rating', ascending=True).head(20)


In [ ]:
ax1 = sns.distplot(filmes_joint['Rating'], norm_hist=True)
ax1.figure.set_size_inches(16, 8)
ax1.set_title('Distribuição das notas', fontsize=18,)
ax1.set_xlabel('Rating', fontsize=14)
ax1=0

In [ ]:
import plotly.express as px

fig = px.box(filmes_joint, x='Rating', orientation= 'h',  title= 'Boxplor das notas IMDb')
fig.show()

In [ ]:
sns.countplot(y="principal_genero",data=filmes_joint)

In [ ]:
filmes_top = filmes_joint.set_index('Title').sort_values('Rating', ascending=False).head(10)
filmes_top

In [ ]:
sns.countplot(y="principal_genero", data=filmes_top)

In [ ]:
filmes_top

In [ ]:
filmes_D = filmes_joint.loc[filmes_joint['votacao_label'] == 'votacao_D']
filmes_C = filmes_joint.loc[filmes_joint['votacao_label'] == 'votacao_C']
filmes_B = filmes_joint.loc[filmes_joint['votacao_label'] == 'votacao_B']
filmes_A = filmes_joint.loc[filmes_joint['votacao_label'] == 'votacao_A']


In [ ]:
sns.set(style="darkgrid")
sns.kdeplot(data=filmes_A['Rating'])

In [ ]:
filmes_A.describe().round(2).T

In [ ]:
filmes_D.describe().round(2).T

In [ ]:
filmes_B.describe().round(2).T

### Curva de densidade da duração de Movies 

- A curva de duração dos Movies se assemelha a uma curva normal, mas assimetrica à direita

In [ ]:
sns.set(style="darkgrid")
sns.kdeplot(data=filmes_joint['duration_movie'])